In [23]:
from bs4 import BeautifulSoup
from collections import Counter 
import re
from torch.utils.data import Dataset, DataLoader
import torch


In [24]:
with open('train_clean.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
train_data = ' '.join(lines)

In [25]:
with open('val_clean.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
val_data = ' '.join(lines)
print(lines[0])

الشّهَادَةِ ظَاهِرَةً ، وَبِحَقّ بَيّنٍ تَضْعُفُ التّهْمَةُ ، وَهُوَ الْفَرْقُ بَيْنَهُ وَبَيْنَ الشّهَادَةِ ، وَعَنْ أَصْبَغَ الْجَوَازُ فِي الْوَلَدِ وَالزّوْجَةِ وَالْأَخِ وَالْمُكَاتَبِ وَالْمُدَبّرِ وَالْمِدْيَانِ إنْ كَانَ مِنْ أَهْلِ الْقِيَامِ بِالْحَقّ ، وَصَحّ الْحُكْمُ ، وَقَدْ يَحْكُمُ لِلْخَلِيفَةِ ، وَهُوَ فَوْقَهُ ، وَتُهْمَتُهُ أَقْوَى ، وَلَا يَنْبَغِي لَهُ الْقَضَاءُ بَيْنَ أَحَدٍ مِنْ عَشِيرَتِهِ وَخَصْمِهِ ، وَإِنْ رَضِيَ الْخَصْمُ بِخِلَافِ رَجُلَيْنِ رَضِيَا بِحُكْمِ رَجُلٍ أَجْنَبِيّ فَيَنْفُذُ ذَلِكَ عَلَيْهِمَا ، وَلَا يَقْضِي بَيْنَهُ وَبَيْنَ غَيْرِهِ ، وَإِنْ رَضِيَ الْخَصْمُ بِذَلِكَ فَإِنْ فَعَلَ فَيُشْهِدُ عَلَى رِضَاهُ ، وَيَجْتَهِدُ فِي الْحَقّ فَإِنْ قَضَى لِنَفْسِهِ أَوْ لِمَنْ يَمْتَنِعُ قَضَاؤُهُ لَهُ فَلْيَذْكُرْ الْقِصّةَ كُلّهَا ، وَرَضِيَ خَصْمِهِ ، وَشَهَادَةَ مَنْ شَهِدَ بِرِضَى الْخَصْمِ



In [26]:
train_data = re.sub(r'\d', '', train_data)
val_data = re.sub(r'\d', '', val_data)

In [27]:
train_data.find('3')

-1

In [28]:
import re

MAX_WORDS = 100      # أقصى عدد كلمات للجملة
MAX_CHARS = 500     # أقصى عدد حروف للجملة

def split_into_sentences(text):
    # Split by sentence delimiters
    sentences = re.split(r"[\.!؟۔،؛\n]+", text)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 0]
    
    result = []
    for sentence in sentences:
        # If sentence is within limits, add it directly
        if len(sentence.split()) <= MAX_WORDS and len(sentence) <= MAX_CHARS:
            result.append(sentence)
        else:
            # Split long sentence into smaller chunks
            result.extend(split_long_sentence(sentence))
    
    return result

def split_long_sentence(sentence):
    """Split a long sentence into smaller chunks based on natural breaks."""
    chunks = []
    
    # Try splitting by commas or semicolons first
    parts = re.split(r'([،,;؛])', sentence)
    
    current_chunk = ""
    for i, part in enumerate(parts):
        # Check if this is a delimiter
        if part in '،,;؛':
            current_chunk += part
            continue
        
        potential_chunk = current_chunk + part
        
        # Check if adding this part exceeds limits
        if (len(potential_chunk.split()) > MAX_WORDS or 
            len(potential_chunk) > MAX_CHARS) and current_chunk:
            # Save current chunk and start new one
            chunks.append(current_chunk.strip())
            current_chunk = part
        else:
            current_chunk = potential_chunk
    
    # Add the last chunk
    if current_chunk.strip():
        chunks.append(current_chunk.strip())
    
    # If chunks are still too long, split by word count
    final_chunks = []
    for chunk in chunks:
        if len(chunk.split()) <= MAX_WORDS and len(chunk) <= MAX_CHARS:
            final_chunks.append(chunk)
        else:
            final_chunks.extend(split_by_words(chunk))
    
    return final_chunks

def split_by_words(text):
    """Split text by word count as a last resort."""
    words = text.split()
    chunks = []
    
    current_chunk = []
    current_length = 0
    
    for word in words:
        potential_length = current_length + len(word) + 1  # +1 for space
        
        if (len(current_chunk) >= MAX_WORDS or 
            potential_length > MAX_CHARS) and current_chunk:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = len(word)
        else:
            current_chunk.append(word)
            current_length = potential_length
    
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks

In [29]:
train_sentences = split_into_sentences(train_data)
val_sentences = split_into_sentences(val_data)

In [30]:
print(len(train_sentences))
print(len(val_sentences))

146083
7339


In [31]:
for i in range(20):
    print(train_sentences[i])

وَلَوْ جَمَعَ ثُمّ عَلِمَ تَرْكَ رُكْنٍ مِنْ الْأُولَى بَطَلَتَا وَيُعِيدُهُمَا جَامِعًا
أَوْ مِنْ الثّانِيَةِ
فَإِنْ لَمْ يَطُلْ تَدَارَكَ
وَإِلّا فَبَاطِلَةٌ وَلَا جَمَعَ
وَلَوْ جَهِلَ أَعَادَهُمَا لِوَقْتَيْهِمَا
قَالَ أَبُو زَيْدٍ أَهْلُ تِهَامَةَ يُؤَنّثُونَ الْعَضُدَ وَبَنُو تَمِيمٍ يُذَكّرُونَ
وَالْجَمْعُ أَعْضُدٌ وَأَعْضَادٌ مِثْلُ أَفْلُسٍ وَأَقْفَالٍ
بِمَنْزِلَةِ أَهْلِ الذّمّةِ إذَا دَخَلُوا قَرْيَةً مِنْ قُرَى أَهْلِ الْحَرْبِ ثُمّ ظَفِرَ الْمُسْلِمُونَ بِهَا فَهُمْ فَيْءٌ أَجْمَعُونَ إلّا مِنْ عُرِفَ أَنّهُ ذِمّيّ
الْمَسْأَلَةُ الْخَامِسَةُ قَوْله تَعَالَى الّذِينَ يُظَاهِرُونَ حَقِيقَتُهُ تَشْبِيهُ ظَهْرٍ بِظَهْرٍ
وَالْمُوجِبُ لِلْحُكْمِ مِنْهُ تَشْبِيهُ ظَهْرِ مُحَلّلٍ بِظَهْرِ مُحَرّمٍ
وَيَتَفَرّعُ عَلَيْهِ فُرُوعٌ كَثِيرَةٌ
أُصُولُهَا سَبْعَةٌ الْفَرْعُ الْأَوّلُ إذَا شَبّهَ جُمْلَةَ أَهْلِهِ بِظَهْرِ أُمّهِ
كَمَا جَاءَ فِي الْحَدِيثِ أَنّهُ قَالَ أَنْتِ عَلَيّ كَظَهْرِ أُمّي
قَوْلُهُ وَاَلّذِي لَا يَتَغَابَنُ النّاسُ فِي مِثْلِهِ مَا لَا يَدْخُلُ تَحْتَ تَقْوِيمِ الْم

In [32]:
ARABIC_DIACRITICS = [
    "َ", "ُ", "ِ", "ْ", "ّ", "َّ", "ُّ", "ِّ", "ً", "ٌ", "ٍ"
]

class Vocab:
    def __init__(self):
        # Input vocab
        self.char2idx_in = {
            '<PAD>': 0,
            '<EOW>': 1,
            '<EOS>':2,
            '<SOS>':3
        }
        self.idx2char_in = {
            0: '<PAD>',
            1: '<EOW>',
            2: '<EOS>',
            3: '<SOS>'
        }

        # Output vocab (diacritics only)
        self.char2idx_out = {
            "": 0,          # No diacritic (الحرف بدون أي تشكيل)
            "َ": 1,         # Fatha  (فتحة)
            "ُ": 2,         # Damma  (ضمة)
            "ِ": 3,         # Kasra  (كسرة)
            "ْ": 4,         # Sukun  (سكون)
            "ّ": 5,         # Shadda (شدة فقط)
            "َّ": 6,        # Shadda + Fatha (شدة + فتحة)
            "ُّ": 7,        # Shadda + Damma (شدة + ضمة)
            "ِّ": 8,        # Shadda + Kasra (شدة + كسرة)
            "ً": 9,         # Tanween Fatha (تنوين فتح)
            "ٌ": 10,        # Tanween Damma (تنوين ضم)
            "ٍ": 11,        # Tanween Kasra (تنوين كسر)
            "<EOW>": 12,     # End Of Word token (علامة نهاية كلمة في إخراج النموذج)
            "<PAD>": 13,     # Padding token (علامة الحشو)
            "<SOS>": 14,
            '<EOS>':15
        }

        self.idx2char_out = {v: k for k, v in self.char2idx_out.items()}

    def build_vocab(self, sentences_raw):

        vocab_input = set()

        for s_raw in sentences_raw:
            for char in s_raw:
                if char.strip(): 
                    vocab_input.add(char)

        vocab_input = sorted(vocab_input)

        # Fill vocab — starting from index 2
        for i, char in enumerate(vocab_input, start=4):
            self.char2idx_in[char] = i
            self.idx2char_in[i] = char

        self.vocab_size_in = len(self.char2idx_in)
        self.vocab_size_out = len(self.char2idx_out)

    def encode_input(self, text):
        out = [self.char2idx_in['<SOS>']]
        for c in text:
            if c == " ":
                out.append(self.char2idx_in['<EOW>'])
            else:
                out.append(self.char2idx_in.get(c, 0))
        out.append(self.char2idx_in['<EOS>'])
        return out

    def decode_input(self, idx_list):
        return ''.join(
            ' ' if i == self.char2idx_in['<EOW>'] else self.idx2char_in.get(i, '')
            for i in idx_list
        )

    def encode_output(self, sentence):
        tokens = [self.char2idx_out['<SOS>']]
        i = 0
        while i < len(sentence):

            char = sentence[i]

            # skip pure diacritic without base
            if char in ARABIC_DIACRITICS:
                i += 1
                continue

            # spaces: no diacritic
            if char == " ":
                tokens.append(12)  # <EOW>
                i += 1
                continue

            diacritics = ""
            j = i + 1

            while j < len(sentence) and sentence[j] in ARABIC_DIACRITICS:
                diacritics += sentence[j]
                j += 1

            # Normalize the diacritic sequence
            # Try to match the key exactly as it appears in char2idx_out
            if diacritics not in self.char2idx_out:
                # If not found, try reversing (sometimes order matters)
                diacritics_reversed = diacritics[::-1]
                if diacritics_reversed in self.char2idx_out:
                    diacritics = diacritics_reversed
                        
            # final diacritic sequence
            tokens.append(self.char2idx_out.get(diacritics, 0))

            i = j
        tokens.append(self.char2idx_out['<EOS>'])
        return tokens

    def decode_output(self, idx):
        return self.idx2char_out.get(idx, "")


In [33]:
vocab = Vocab()

In [34]:
def remove_diacritics(sentences):
    no_diac_senteces = []
    for sentence in sentences:
         chars = [char for char in sentence if char not in ARABIC_DIACRITICS]
         no_diac_senteces.append(''.join(chars))
    return no_diac_senteces

In [35]:
no_diac_sentences_train = remove_diacritics(train_sentences)
no_diac_sentences_val = remove_diacritics(val_sentences)

In [36]:
vocab.build_vocab(no_diac_sentences_train)

In [37]:
print(no_diac_sentences_train[0])
print(no_diac_sentences_val[0])


ولو جمع ثم علم ترك ركن من الأولى بطلتا ويعيدهما جامعا
الشهادة ظاهرة


In [38]:
def test_all_diacritics():
    
    test_cases = [
        # Test case format: (input_text, expected_output, description)
        
        # 1. No diacritic
        ("الكتاب", [0, 0, 0, 0, 0, 0], "No diacritics"),
        
        # 2. Fatha (َ)
        ("كَتَبَ", [1, 1, 1], "Fatha only"),
        
        # 3. Damma (ُ)
        ("كُتُبُ", [2, 2, 2], "Damma only"),
        
        # 4. Kasra (ِ)
        ("كِتِبِ", [3, 3, 3], "Kasra only"),
        
        # 5. Sukun (ْ)
        ("كْتْبْ", [4, 4, 4], "Sukun only"),
        
        # 6. Shadda alone (ّ)
        ("كّتّبّ", [5, 5, 5], "Shadda alone"),
        
        # 7. Shadda + Fatha (َّ)
        ("كَّتَّبَّ", [6, 6, 6], "Shadda + Fatha"),
        
        # 8. Shadda + Damma (ُّ)
        ("كُّتُّبُّ", [7, 7, 7], "Shadda + Damma"),
        
        # 9. Shadda + Kasra (ِّ)
        ("كِّتِّبِّ", [8, 8, 8], "Shadda + Kasra"),
        
        # 10. Tanween Fatha (ً)
        ("كتابًا", [0, 0, 0, 0, 9, 0], "Tanween Fatha"),
        
        # 11. Tanween Damma (ٌ)
        ("كتابٌ", [0, 0, 0, 0, 10], "Tanween Damma"),
        
        # 12. Tanween Kasra (ٍ)
        ("كتابٍ", [0, 0, 0, 0, 11], "Tanween Kasra"),
        
        # 13. Mixed diacritics
        ("كَتَبَ الْوَلَدُ", [1, 1, 1, 12, 0, 4, 1, 1, 2], "Mixed diacritics with space"),
        
        # 14. Real word: السَّلَامُ
        ("السَّلَامُ", [0, 0, 6, 0, 0, 2], "As-salamu (with Shadda+Fatha)"),
        
        # 15. Real word: مُحَمَّدٌ
        ("مُحَمَّدٌ", [2, 1, 1, 6, 10], "Muhammad (with Shadda+Fatha and Tanween Damma)"),
        
        # 16. Real word: بِسْمِ اللَّهِ
        ("بِسْمِ اللَّهِ", [3, 4, 3, 12, 0, 0, 6, 3], "Bismillah"),
        
        # 17. Complex: all diacritics in sequence
        ("بَبُبِبْبّبَّبُّبِّبًبٌبٍ", [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], "All diacritics on same letter"),
        
        # 18. Word with spaces
        ("كَتَبَ وَلَدٌ", [1, 1, 1, 12, 1, 1, 10], "Two words with space"),
        
        # 19. Empty/edge cases
        ("", [], "Empty string"),
        ("   ", [12, 12, 12], "Only spaces"),
        
        # 20. Letters without diacritics between diacritized ones
        ("كَتبُ", [1, 0, 0, 2], "Mixed diacritized and non-diacritized"),
    ]
    
    print("Testing all diacritics...\n")
    passed = 0
    failed = 0
    
    for i, (input_text, expected, description) in enumerate(test_cases, 1):
        result = vocab.encode_output(input_text)
        
        if result == expected:
            print(f"✓ Test {i}: {description}")
            print(f"  Input: '{input_text}'")
            print(f"  Output: {result}")
            passed += 1
        else:
            print(f"✗ Test {i}: {description}")
            print(f"  Input: '{input_text}'")
            print(f"  Expected: {expected}")
            print(f"  Got:      {result}")
            failed += 1
        print()
    
    print(f"\n{'='*50}")
    print(f"Results: {passed} passed, {failed} failed out of {len(test_cases)} tests")
    print(f"{'='*50}")
    
    return passed, failed


# Run the tests
if __name__ == "__main__":
    test_all_diacritics()

Testing all diacritics...

✗ Test 1: No diacritics
  Input: 'الكتاب'
  Expected: [0, 0, 0, 0, 0, 0]
  Got:      [14, 0, 0, 0, 0, 0, 0, 15]

✗ Test 2: Fatha only
  Input: 'كَتَبَ'
  Expected: [1, 1, 1]
  Got:      [14, 1, 1, 1, 15]

✗ Test 3: Damma only
  Input: 'كُتُبُ'
  Expected: [2, 2, 2]
  Got:      [14, 2, 2, 2, 15]

✗ Test 4: Kasra only
  Input: 'كِتِبِ'
  Expected: [3, 3, 3]
  Got:      [14, 3, 3, 3, 15]

✗ Test 5: Sukun only
  Input: 'كْتْبْ'
  Expected: [4, 4, 4]
  Got:      [14, 4, 4, 4, 15]

✗ Test 6: Shadda alone
  Input: 'كّتّبّ'
  Expected: [5, 5, 5]
  Got:      [14, 5, 5, 5, 15]

✗ Test 7: Shadda + Fatha
  Input: 'كَّتَّبَّ'
  Expected: [6, 6, 6]
  Got:      [14, 6, 6, 6, 15]

✗ Test 8: Shadda + Damma
  Input: 'كُّتُّبُّ'
  Expected: [7, 7, 7]
  Got:      [14, 7, 7, 7, 15]

✗ Test 9: Shadda + Kasra
  Input: 'كِّتِّبِّ'
  Expected: [8, 8, 8]
  Got:      [14, 8, 8, 8, 15]

✗ Test 10: Tanween Fatha
  Input: 'كتابًا'
  Expected: [0, 0, 0, 0, 9, 0]
  Got:      [14, 0, 0, 0, 9

In [39]:
print(vocab.encode_input('ولو جمع'))
vocab.encode_output("السَّلَامُ")

[3, 37, 33, 37, 1, 15, 34, 28, 2]


[14, 0, 0, 6, 1, 0, 2, 15]

In [40]:
print(len(vocab.char2idx_in))
print(vocab.char2idx_in)

40
{'<PAD>': 0, '<EOW>': 1, '<EOS>': 2, '<SOS>': 3, 'ء': 4, 'آ': 5, 'أ': 6, 'ؤ': 7, 'إ': 8, 'ئ': 9, 'ا': 10, 'ب': 11, 'ة': 12, 'ت': 13, 'ث': 14, 'ج': 15, 'ح': 16, 'خ': 17, 'د': 18, 'ذ': 19, 'ر': 20, 'ز': 21, 'س': 22, 'ش': 23, 'ص': 24, 'ض': 25, 'ط': 26, 'ظ': 27, 'ع': 28, 'غ': 29, 'ف': 30, 'ق': 31, 'ك': 32, 'ل': 33, 'م': 34, 'ن': 35, 'ه': 36, 'و': 37, 'ى': 38, 'ي': 39}


In [41]:
def create_windows(sentence, window_size=9):
    words = sentence.split()
    windows = []
    for i in range(len(words) - window_size + 1):
        windows.append(' '.join(words[i:i + window_size]))
    return windows

In [42]:
window_sentences_with_diacritics_train = []
for sentence in train_sentences:
    windows = create_windows(sentence, window_size=9)
    window_sentences_with_diacritics_train.extend(windows)

In [43]:
window_sentences_with_no_diacritics_train = []
for sentence in no_diac_sentences_train:
    windows = create_windows(sentence, window_size=9)
    window_sentences_with_no_diacritics_train.extend(windows)

In [44]:
window_sentences_with_diacritics_val = []
for sentence in val_sentences:
    windows = create_windows(sentence, window_size=9)
    window_sentences_with_diacritics_val.extend(windows)

In [45]:
window_sentences_with_no_diacritics_val = []
for sentence in no_diac_sentences_val:
    windows = create_windows(sentence, window_size=9)
    window_sentences_with_no_diacritics_val.extend(windows)

In [46]:
print(vocab.char2idx_out)

{'': 0, 'َ': 1, 'ُ': 2, 'ِ': 3, 'ْ': 4, 'ّ': 5, 'َّ': 6, 'ُّ': 7, 'ِّ': 8, 'ً': 9, 'ٌ': 10, 'ٍ': 11, '<EOW>': 12, '<PAD>': 13, '<SOS>': 14, '<EOS>': 15}


In [47]:
window_sentences_with_diacritics_train[0]

'وَلَوْ جَمَعَ ثُمّ عَلِمَ تَرْكَ رُكْنٍ مِنْ الْأُولَى بَطَلَتَا'

In [48]:
print(len(window_sentences_with_diacritics_val))
print(len(window_sentences_with_no_diacritics_val))
print(len(window_sentences_with_diacritics_train))
print(len(window_sentences_with_no_diacritics_train))

55094
55094
1154746
1154746


In [49]:
print(window_sentences_with_diacritics_train[0])

وَلَوْ جَمَعَ ثُمّ عَلِمَ تَرْكَ رُكْنٍ مِنْ الْأُولَى بَطَلَتَا


In [50]:
print(len(vocab.encode_input(window_sentences_with_no_diacritics_train[2])))
print(len(vocab.encode_output(window_sentences_with_diacritics_train[2])))

47
47


In [51]:
class DiacritizationDataset(Dataset):
    def __init__(self, sentences_raw, sentences_diac,vocab):
        self.data = []

        for s_raw, s_diac in zip(sentences_raw, sentences_diac):
                input_tokens = vocab.encode_input(s_raw) 
                output_tokens = vocab.encode_output(s_diac)

                if len(input_tokens) == len(output_tokens): 
                    self.data.append((input_tokens, output_tokens))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.tensor(x), torch.tensor(y)

def collate_fn(batch):
    xs, ys = zip(*batch)
    xs = torch.nn.utils.rnn.pad_sequence(xs, batch_first=True, padding_value=0) 
    ys = torch.nn.utils.rnn.pad_sequence(ys, batch_first=True, padding_value=13) # Padding value for output is 13
    return xs, ys


In [52]:
dataset_train = DiacritizationDataset(window_sentences_with_no_diacritics_train, window_sentences_with_diacritics_train, vocab)
train_loader = DataLoader(dataset_train, batch_size=32, collate_fn=collate_fn, shuffle=True)

In [53]:
dataset_val = DiacritizationDataset(window_sentences_with_no_diacritics_val, window_sentences_with_diacritics_val, vocab)
val_loader = DataLoader(dataset_val, batch_size=32, collate_fn=collate_fn, shuffle=False)

In [54]:
vocab.encode_output("السَّلَامُ عَلَيْكُمْ")

[14, 0, 0, 6, 1, 0, 2, 12, 1, 1, 4, 2, 4, 15]

In [55]:
def print_first_batch(dataloader):
    """Print the first batch from dataloader."""
    for batch in dataloader:
        print("First Batch:")
        print("="*50)
        
        # If batch is a dictionary
        if isinstance(batch, dict):
            for key, value in batch.items():
                print(f"\n{key}:")
                print(f"  Shape: {value.shape}")
                print(f"  Data: {value}")
        
        # If batch is a tuple/list
        elif isinstance(batch, (tuple, list)):
            for i, item in enumerate(batch):
                print(f"\nItem {i}:")
                print(f"  Shape: {item.shape}")
                print(f"  Data: {item}")
        
        # If batch is a single tensor
        else:
            print(f"Shape: {batch.shape}")
            print(f"Data: {batch}")
        
        break  # Stop after first batch

print_first_batch(train_loader)

First Batch:

Item 0:
  Shape: torch.Size([32, 52])
  Data: tensor([[ 3, 30, 39,  ..., 35,  2,  0],
        [ 3, 33,  6,  ..., 39,  2,  0],
        [ 3,  6, 37,  ...,  0,  0,  0],
        ...,
        [ 3, 11, 35,  ...,  0,  0,  0],
        [ 3, 11, 35,  ...,  0,  0,  0],
        [ 3, 24, 33,  ...,  0,  0,  0]])

Item 1:
  Shape: torch.Size([32, 52])
  Data: tensor([[14,  3,  0,  ...,  3, 15, 13],
        [14,  3,  1,  ...,  0, 15, 13],
        [14,  1,  4,  ..., 13, 13, 13],
        ...,
        [14,  4,  3,  ..., 13, 13, 13],
        [14,  4,  3,  ..., 13, 13, 13],
        [14,  1,  5,  ..., 13, 13, 13]])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import os

# ==================== MODEL COMPONENTS ====================

class Encoder(nn.Module):
    def __init__(self, input_vocab_size, embed_size, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(input_vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, 
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # x: [batch_size, seq_len]
        embedded = self.dropout(self.embedding(x))  # [batch_size, seq_len, embed_size]
        
        # outputs: [batch_size, seq_len, hidden_size]
        # hidden: [num_layers, batch_size, hidden_size]
        # cell: [num_layers, batch_size, hidden_size]
        outputs, (hidden, cell) = self.lstm(embedded)
        
        return outputs, hidden, cell


class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.hidden_size = hidden_size
        
        # Attention weights
        self.W1 = nn.Linear(hidden_size, hidden_size)  # For encoder outputs
        self.W2 = nn.Linear(hidden_size, hidden_size)  # For decoder hidden state
        self.V = nn.Linear(hidden_size, 1)  # Final attention score
    
    def forward(self, encoder_outputs, decoder_hidden):
        # encoder_outputs: [batch_size, seq_len, hidden_size]
        # decoder_hidden: [batch_size, hidden_size]
        
        # Add time dimension to decoder_hidden
        decoder_hidden = decoder_hidden.unsqueeze(1)  # [batch_size, 1, hidden_size]
        
        # Calculate attention scores
        # [batch_size, seq_len, hidden_size]
        energy = torch.tanh(self.W1(encoder_outputs) + self.W2(decoder_hidden))
        
        # [batch_size, seq_len, 1]
        attention_scores = self.V(energy)
        
        # [batch_size, seq_len]
        attention_weights = F.softmax(attention_scores.squeeze(2), dim=1)
        
        # Context vector: weighted sum of encoder outputs
        # [batch_size, 1, seq_len] x [batch_size, seq_len, hidden_size]
        # = [batch_size, 1, hidden_size]
        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs)
        
        # [batch_size, hidden_size]
        context_vector = context_vector.squeeze(1)
        
        return context_vector, attention_weights


class Decoder(nn.Module):
    def __init__(self, output_vocab_size, embed_size, hidden_size, num_layers, dropout):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_vocab_size = output_vocab_size
        
        self.embedding = nn.Embedding(output_vocab_size, embed_size, padding_idx=0)
        self.attention = BahdanauAttention(hidden_size)
        
        # LSTM input: embedding + context vector
        self.lstm = nn.LSTM(embed_size + hidden_size, hidden_size, num_layers,
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)
        
        # Final output layer
        self.fc = nn.Linear(hidden_size, output_vocab_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, encoder_outputs, hidden, cell):
        # x: [batch_size, 1] - single time step
        # encoder_outputs: [batch_size, seq_len, hidden_size]
        # hidden: [num_layers, batch_size, hidden_size]
        # cell: [num_layers, batch_size, hidden_size]
        
        # Embed input
        embedded = self.dropout(self.embedding(x))  # [batch_size, 1, embed_size]
        
        # Get attention context using top decoder hidden state
        context, attention_weights = self.attention(encoder_outputs, hidden[-1])
        # context: [batch_size, hidden_size]
        
        # Concatenate embedded input and context
        # [batch_size, 1, embed_size + hidden_size]
        lstm_input = torch.cat([embedded, context.unsqueeze(1)], dim=2)
        
        # LSTM step
        output, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))
        # output: [batch_size, 1, hidden_size]
        
        # Prediction
        prediction = self.fc(output.squeeze(1))  # [batch_size, output_vocab_size]
        
        return prediction, hidden, cell, attention_weights


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [batch_size, src_len]
        # trg: [batch_size, trg_len]
        
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_vocab_size
        
        # Store outputs
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        
        # Encode
        encoder_outputs, hidden, cell = self.encoder(src)
        
        # First input to decoder is <SOS> token (index 1)
        decoder_input = trg[:, 0].unsqueeze(1)  # [batch_size, 1]
        
        # Decode
        for t in range(1, trg_len):
            # Get prediction
            prediction, hidden, cell, _ = self.decoder(
                decoder_input, encoder_outputs, hidden, cell
            )
            
            # Store prediction
            outputs[:, t] = prediction
            
            # Teacher forcing
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            
            # Get next input
            top1 = prediction.argmax(1)
            decoder_input = trg[:, t].unsqueeze(1) if teacher_force else top1.unsqueeze(1)
        
        return outputs


# ==================== MODEL LOADING FUNCTION ====================

def load_model_if_exists(model, optimizer, save_path, device):
    """
    Load model checkpoint if it exists.
    
    Returns:
        start_epoch: Epoch to resume training from (0 if no checkpoint)
        best_val_loss: Best validation loss from checkpoint (inf if no checkpoint)
    """
    if os.path.exists(save_path):
        print(f"Loading checkpoint from {save_path}...")
        checkpoint = torch.load(save_path, map_location=device)
        
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
        start_epoch = checkpoint['epoch'] + 1
        best_val_loss = checkpoint['val_loss']
        
        print(f"✓ Checkpoint loaded successfully!")
        print(f"  - Resuming from epoch {start_epoch}")
        print(f"  - Best validation loss: {best_val_loss:.4f}")
        print(f"  - Previous train loss: {checkpoint['train_loss']:.4f}")
        
        return start_epoch, best_val_loss
    else:
        print(f"No checkpoint found at {save_path}. Starting from scratch.")
        return 0, float('inf')


# ==================== TRAINING FUNCTION ====================

def train_epoch(model, dataloader, optimizer, criterion, device, clip=1.0):
    model.train()
    epoch_loss = 0
    
    progress_bar = tqdm(dataloader, desc="Training")
    
    for batch_idx, batch in enumerate(progress_bar):
        src = batch[0].to(device)  # Input (undiacritized)
        trg = batch[1].to(device)  # Target (diacritized)
        
        optimizer.zero_grad()
        
        # Forward pass
        output = model(src, trg, teacher_forcing_ratio=0.5)
        
        # Reshape for loss calculation
        # output: [batch_size, trg_len, output_vocab_size]
        # trg: [batch_size, trg_len]
        
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)  # Skip <SOS>
        trg = trg[:, 1:].reshape(-1)
        
        # Calculate loss (ignore padding)
        loss = criterion(output, trg)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        # Update progress bar
        progress_bar.set_postfix({'loss': loss.item()})
    
    return epoch_loss / len(dataloader)


def evaluate(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            src = batch[0].to(device)
            trg = batch[1].to(device)
            
            # Turn off teacher forcing
            output = model(src, trg, teacher_forcing_ratio=0)
            
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)
            
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    
    return epoch_loss / len(dataloader)


# ==================== TRAINING LOOP ====================

def train_model(model, train_loader, val_loader, optimizer, criterion, 
                device, num_epochs=10, clip=1.0, save_path='best_model.pt'):
    
    # Load checkpoint if exists
    start_epoch, best_val_loss = load_model_if_exists(model, optimizer, save_path, device)
    
    train_losses = []
    val_losses = []
    
    for epoch in range(start_epoch, num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 50)
        
        # Train
        train_loss = train_epoch(model, train_loader, optimizer, criterion, device, clip)
        train_losses.append(train_loss)
        
        # Validate
        val_loss = evaluate(model, val_loader, criterion, device)
        val_losses.append(val_loss)
        
        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
            }, 'best_diacritization_model.pt')
            print(f"✓ Model saved with val_loss: {val_loss:.4f}")
    
    return train_losses, val_losses


# ==================== USAGE EXAMPLE ====================

if __name__ == "__main__":
    # Hyperparameters (from paper)
    INPUT_VOCAB_SIZE = vocab.vocab_size_in  # Your input vocab size
    OUTPUT_VOCAB_SIZE = vocab.vocab_size_out  # Your output vocab size
    EMBED_SIZE = 512
    HIDDEN_SIZE = 512
    NUM_LAYERS = 2
    DROPOUT = 0.3
    LEARNING_RATE = 0.001
    NUM_EPOCHS = 20
    CLIP = 1.0
    
    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Initialize model
    encoder = Encoder(INPUT_VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
    decoder = Decoder(OUTPUT_VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
    model = Seq2Seq(encoder, decoder, device).to(device)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total trainable parameters: {total_params:,}")
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss(ignore_index=13)  # Ignore padding
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Train (will automatically load checkpoint if exists)
    train_losses, val_losses = train_model(
        model, train_loader, val_loader, optimizer, criterion,
        device, NUM_EPOCHS, CLIP, save_path='best_diacritization_model.pt'
    )
    
    # Plot losses
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.savefig('training_losses.png')
    plt.show()

Using device: cuda
Total trainable parameters: 10,016,273
No checkpoint found at best_diacritization_mdel.pt. Starting from scratch.

Epoch 1/20
--------------------------------------------------


Training:   0%|          | 27/36086 [00:04<1:37:45,  6.15it/s, loss=1.84]

Evaluating: 100%|██████████| 1722/1722 [01:10<00:00, 24.46it/s]


Train Loss: 0.2134 | Val Loss: 0.1099
✓ Model saved with val_loss: 0.1099

Epoch 2/20
--------------------------------------------------


Evaluating: 100%|██████████| 1722/1722 [01:10<00:00, 24.40it/s]


...

Evaluating: 100%|██████████| 1722/1722 [01:11<00:00, 24.12it/s]


Train Loss: 0.0713 | Val Loss: 0.0833
✓ Model saved with val_loss: 0.0833

Epoch 6/20
--------------------------------------------------


Evaluating: 100%|██████████| 1722/1722 [01:11<00:00, 24.24it/s]


Train Loss: 0.0695 | Val Loss: 0.0857

Epoch 7/20
--------------------------------------------------


Training:  54%|█████▍    | 19661/36086 [54:34<46:52,  5.84it/s, loss=0.0837]  

In [ ]:
dataset_val_no_window = DiacritizationDataset(val_sentences, no_diac_sentences_val, vocab)
val_loader_no_window = DataLoader(dataset_val, batch_size=32, collate_fn=collate_fn, shuffle=False)

In [ ]:
def calculate_character_accuracy(model, dataloader, device):
    """Calculate character-level accuracy (DER - Diacritic Error Rate)"""
    model.eval()

    total_chars = 0
    correct_chars = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Calculating Accuracy"):
            src = batch[0].to(device) # (batch_size,seq_length)
            trg = batch[1].to(device) # (batch_size,seq_length)

            # Get predictions (no teacher forcing)
            output = model(src, trg, teacher_forcing_ratio=0)

            # Get predicted tokens
            predictions = output.argmax(2)  # [batch_size, seq_len]

            # Compare with targets (skip <SOS> token at position 0)
            for i in range(predictions.shape[0]):
                pred_seq = predictions[i, 1:]  # Skip <SOS>
                target_seq = trg[i, 1:]  # Skip <SOS>

                # Only compare non-padding tokens
                mask = target_seq != 13  # Not padding

                if mask.sum() > 0:
                    valid_preds = pred_seq[mask]
                    valid_targets = target_seq[mask]
                    # Count correct predictions
                    correct_chars += (valid_preds == valid_targets).sum().item()
                    total_chars += len(valid_targets)

    accuracy = (correct_chars / total_chars) * 100 if total_chars > 0 else 0
    error_rate = 100 - accuracy

    return accuracy, error_rate


In [ ]:
# ==================== LOAD MODEL AND EVALUATE ====================

def load_and_evaluate(model_path, val_loader, vocab):
    """Load trained model and calculate accuracy metrics"""

    # Hyperparameters (must match training)
    INPUT_VOCAB_SIZE = vocab.vocab_size_in
    OUTPUT_VOCAB_SIZE = vocab.vocab_size_out
    EMBED_SIZE = 512
    HIDDEN_SIZE = 512
    NUM_LAYERS = 2
    DROPOUT = 0.3

    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}\n")

    # Initialize model
    encoder = Encoder(INPUT_VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
    decoder = Decoder(OUTPUT_VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
    model = Seq2Seq(encoder, decoder, device).to(device)

    # Load checkpoint
    print(f"Loading model from: {model_path}")
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])

    print(f"Model loaded successfully!")
    print(f"Training epoch: {checkpoint['epoch']}")
    print(f"Training loss: {checkpoint['train_loss']:.4f}")
    print(f"Validation loss: {checkpoint['val_loss']:.4f}\n")

    # Calculate metrics
    print("=" * 60)
    print("Calculating Accuracy Metrics...")
    print("=" * 60)

    char_accuracy, der = calculate_character_accuracy(model, val_loader, device)
    # word_accuracy, wer = calculate_word_accuracy(model, val_loader, device)

    # Print results
    print("\n" + "=" * 60)
    print("EVALUATION RESULTS")
    print("=" * 60)
    print(f"Character Accuracy: {char_accuracy:.2f}%")
    print(f"DER (Diacritic Error Rate): {der:.2f}%")
    # print(f"\nWord Accuracy: {word_accuracy:.2f}%")
    # print(f"WER (Word Error Rate): {wer:.2f}%")
    print("=" * 60)

    return {
        'char_accuracy': char_accuracy,
        'der': der,
        # 'word_accuracy': word_accuracy,
        # 'wer': wer
    }

In [ ]:
model_path = 'best_diacritization_model.pt'

# Load and evaluate
results = load_and_evaluate(
    model_path=model_path,
    val_loader=val_loader_no_window,  # Your validation dataloader
    vocab = vocab
)

In [ ]:
def predict_sentence(model, sentence, vocab, device, max_len=300):
    """
    Predict diacritics for a single sentence.

    Args:
        model: Trained Seq2Seq model
        sentence: Input sentence (without diacritics)
        device: torch device
        max_len: Maximum sequence length

    Returns:
        diacritized_sentence: Sentence with predicted diacritics
    """
    model.eval()

    input_tokens = vocab.encode_input(sentence)
    # Convert to tensor
    src = torch.tensor([input_tokens]).to(device)  # [1, seq_len]

    with torch.no_grad():
        # Encode
        encoder_outputs, hidden, cell = model.encoder(src)

        # Start decoding with <SOS> token
        decoder_input = torch.tensor([[vocab.char2idx_out['<SOS>']]]).to(device) # [1,1]

        predictions = []

        # Decode step by step
        for _ in range(len(input_tokens) - 2):  # -2 for <SOS> and <EOS>
            prediction, hidden, cell, _ = model.decoder(
                decoder_input, encoder_outputs, hidden, cell
            )

            # Get most likely token
            top1 = prediction.argmax(1)
            predicted_token = top1.item()

            # Stop if <EOS> or <PAD>
            if predicted_token == vocab.char2idx_out['<EOS>']:
                print('end of the sentence')
                break
            predictions.append(predicted_token)

            # Next input
            decoder_input = top1.unsqueeze(0)

        # Decode predictions back to text
        # diacritized_sentence = [vocab.idx2char_out[idx] for idx in predictions]
        diacritized_sentence = [idx for idx in predictions]


    return diacritized_sentence

def test_single_sentence(sentence, model_path, vocab):
    """Test a single sentence and return result"""

    # Hyperparameters
    INPUT_VOCAB_SIZE = vocab.vocab_size_in
    OUTPUT_VOCAB_SIZE = vocab.vocab_size_out
    EMBED_SIZE = 512
    HIDDEN_SIZE = 512
    NUM_LAYERS = 2
    DROPOUT = 0.3

    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load model
    encoder = Encoder(INPUT_VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
    decoder = Decoder(OUTPUT_VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
    model = Seq2Seq(encoder, decoder, device).to(device)

    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    sentence.replace(' ','<EOW>')
    # Predict
    diacritized = predict_sentence(
        model, sentence, vocab, device
    )
    diacritized.reverse()
    print("=" * 60)
    print("DIACRITIZATION RESULT")
    print("=" * 60)
    print(len(sentence))
    print(len(diacritized))
    print(f"Input:  {sentence}")
    print(f"Output: {diacritized}")
    print("=" * 60)

    return diacritized

In [ ]:
def load_model(model_path, vocab):
    """Load trained model and calculate accuracy metrics"""

    # Hyperparameters (must match training)
    INPUT_VOCAB_SIZE = vocab.vocab_size_in
    OUTPUT_VOCAB_SIZE = vocab.vocab_size_out
    EMBED_SIZE = 512
    HIDDEN_SIZE = 512
    NUM_LAYERS = 2
    DROPOUT = 0.3

    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}\n")

    # Initialize model
    encoder = Encoder(INPUT_VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
    decoder = Decoder(OUTPUT_VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
    model = Seq2Seq(encoder, decoder, device).to(device)

    # Load checkpoint
    print(f"Loading model from: {model_path}")
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])

    print(f"Model loaded successfully!")
    print(f"Training epoch: {checkpoint['epoch']}")
    print(f"Training loss: {checkpoint['train_loss']:.4f}")
    print(f"Validation loss: {checkpoint['val_loss']:.4f}\n")

    return model

In [ ]:
def create_sliding_windows(sentence, window_size=3):
    """
    Create sliding windows of consecutive words from a sentence.
    For sentence "w1 w2 w3 w4 w5" with window_size=3:
    Returns: ["w1 w2 w3", "w2 w3 w4", "w3 w4 w5"]
    Also returns the word lengths for each window.
    """
    words = sentence.split()
    windows = [] 
    window_word_lengths = []  # Store character lengths of words in each window

    if len(words) <= window_size:
        # If sentence is shorter than or equal to window size, return entire sentence
        word_lengths = [len(word) for word in words]
        return [sentence], [word_lengths]

    # Create sliding windows
    for i in range(len(words) - window_size + 1):
        window_words = words[i:i + window_size]
        window = ' '.join(window_words)
        windows.append(window)

        # Store the character length of each word in this window
        word_lengths = [len(word) for word in window_words]
        window_word_lengths.append(word_lengths)
    print(word_lengths)
    return windows, window_word_lengths


def predict_window(model, window_text, word_lengths,vocab, device):
    """
    Predict diacritics for a single window.
    Returns list of predicted words with diacritics.

    Args:
        model: Trained Seq2Seq model
        window_text: String of words in the window (without diacritics)
        word_lengths: List of character lengths for each word in window
        vocab_input: Input vocabulary list
        vocab_output: Output vocabulary list
        device: torch device

    Returns:
        List of predicted words with diacritics
    """
    model.eval()

    input_tokens = vocab.encode_input(window_text)

    src = torch.tensor([input_tokens]).to(device)

    with torch.no_grad():
        # Encode
        encoder_outputs, hidden, cell = model.encoder(src)

        # Start decoding
        sos_idx = vocab.char2idx_out['<SOS>']
        decoder_input = torch.tensor([[sos_idx]]).to(device) # (1,1)

        predictions = []
        eos_idx = vocab.char2idx_out['<EOS>']

        # Predict characters (output will have diacritics, so potentially longer)
        # max_length = len(input_text_no_spaces) * 3

        for step in range(len(input_tokens) - 2):
            prediction, hidden, cell, _ = model.decoder(
                decoder_input, encoder_outputs, hidden, cell
            )

            top1 = prediction.argmax(1)
            predicted_token = top1.item()

            # Check for EOS token
            if predicted_token == eos_idx:
                break

            predictions.append(predicted_token)
            decoder_input = top1.unsqueeze(0)

        # Decode to text (continuous string with diacritics, no spaces)
        diacritized_text = predictions
        # diacritized_text = [vocab.idx2char_out.get(idx, '') for idx in predictions]

        # Clean up tokens
        # diacritized_text = [' ' if c == '<EOW>' else c for c in diacritized_text]

    # Now split the predicted text into words using the original word lengths
    print(f"Predicted diacritized text tokens: {diacritized_text}")
    predicted_words = []
    char_idx = 0
    for word_len in word_lengths:
        # For each original word, we need to extract characters from predicted text
        # The predicted text has diacritics, so we need to count base characters

        word_diacritics = []
        for _ in range(word_len):
            word_diacritics.append(diacritized_text[char_idx])
            char_idx += 1
        word_diacritics
        char_idx += 1 # for space
        predicted_words.append(word_diacritics)

    predicted_words
    return predicted_words


def predict_sentence_with_voting(model, sentence, vocab, device, window_size=9):
    model.eval()

    # Step 1: Split into words and store lengths
    words = sentence.split()
    num_words = len(words)
    word_lengths = [len(word) for word in words]

    if num_words == 0:
        return ""

    print(f"Input: {num_words} words")
    print(f"Word lengths: {word_lengths}")
    print(f"Words: {words}")
    print()

    # Handle short sentences
    if num_words <= window_size:
        print(f"Sentence has {num_words} words, window size is {window_size}")
        print("Processing entire sentence as one window")
        try:
            predicted_words = predict_window(model, sentence, word_lengths, vocab, device)
        except Exception as e:
            print(f"Error predicting sentence: {e}")
            import traceback
            traceback.print_exc()
            return sentence

    # Step 2: Create sliding windows
    windows, window_word_lengths = create_sliding_windows(sentence, window_size)

    print(f"Created {len(windows)} windows\n")

    # Step 3: Collect predictions
    word_predictions = [[] for _ in range(num_words)]

    for window_idx, (window, w_lengths) in enumerate(zip(windows, window_word_lengths)):
        try:
            print(f"Window {window_idx}: {window}")

            predicted_words = predict_window(model, window, w_lengths, vocab, device)
            print(f"Predicted words: {predicted_words}")

            window_start_idx = window_idx

            for i, pred_word in enumerate(predicted_words):
                actual_idx = window_start_idx + i
                if 0 <= actual_idx < num_words:
                    # Keep pred_word as LIST OF DIACRITICS (not string)
                    word_predictions[actual_idx].append(pred_word)
                    print(f"  -> Stored for word {actual_idx}")

            print()

        except Exception as e:
            print(f"Error in window {window_idx}: {e}")
            import traceback
            traceback.print_exc()
            window_start_idx = window_idx
            for i in range(min(window_size, num_words - window_idx)):
                actual_idx = window_start_idx + i
                if 0 <= actual_idx < num_words:
                    word_predictions[actual_idx].append(list(words[actual_idx]))

    # Step 4: Voting (modified)
    final_words = []

    print("="*60)
    print("VOTING RESULTS:")
    print("="*60)

    from collections import Counter

    for word_idx, predictions in enumerate(word_predictions):
        if len(predictions) == 0:
            print(f"Word {word_idx} ({words[word_idx]}): No predictions -> using original")
            final_words.append(list(words[word_idx]))
            continue

        if len(predictions) == 1:
            print(f"Word {word_idx} ({words[word_idx]}): 1 prediction -> {predictions[0]}")
            final_words.append(predictions[0])
            continue

        # --- Character-level voting ---
        word_len = len(words[word_idx])
        final_word = []

        for pos in range(word_len):
            chars_at_pos = []
            for pred in predictions:
                if len(pred) > pos:
                    chars_at_pos.append(pred[pos])

            if len(chars_at_pos) > 0:
                best_char = Counter(chars_at_pos).most_common(1)[0][0]
            else:
                # fallback: use original character (no diacritic)
                best_char = words[word_idx][pos]

            final_word.append(best_char)

        print(f"Word {word_idx} ({words[word_idx]}): {len(predictions)} predictions -> {final_word}")
        final_words.append(final_word)

    print("="*60)

    return final_words



# Expected behavior for 5 words with window_size=3:
# Window 0: w0 w1 w2 -> predictions for words 0, 1, 2
# Window 1: w1 w2 w3 -> predictions for words 1, 2, 3
# Window 2: w2 w3 w4 -> predictions for words 2, 3, 4
#
# Voting:
# word_predictions[0] = [pred from window 0] -> 1 prediction
# word_predictions[1] = [pred from window 0, pred from window 1] -> 2 predictions (vote)
# word_predictions[2] = [pred from window 0, pred from window 1, pred from window 2] -> 3 predictions (vote)
# word_predictions[3] = [pred from window 1, pred from window 2] -> 2 predictions (vote)
# word_predictions[4] = [pred from window 2] -> 1 prediction

In [ ]:
model_path = 'best_diacritization_model.pt'
model = load_model(model_path,vocab)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
final_words = predict_sentence_with_voting(model,'ذهبت الى الحديقة مع اصدقاء السوء لتركب العجلة',vocab,device,window_size=5)
final_words_reversed = [] 
for word in final_words:
    word.reverse()
    final_words_reversed.append(word)
final_words_reversed

In [ ]:
def calculate_last_character_accuracy(model, dataloader, device):
    """Calculate accuracy for the last character of each word (before EOW token)"""
    model.eval()

    total_last_chars = 0
    correct_last_chars = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Calculating Last Char Accuracy"):
            src = batch[0].to(device)  # (batch_size, seq_length)
            trg = batch[1].to(device)  # (batch_size, seq_length)

            # Get predictions (no teacher forcing)
            output = model(src, trg, teacher_forcing_ratio=0)

            # Get predicted tokens
            predictions = output.argmax(2)  # [batch_size, seq_len]

            # Compare with targets (skip <SOS> token at position 0)
            for i in range(predictions.shape[0]):
                pred_seq = predictions[i, 1:]  # Skip <SOS>
                target_seq = trg[i, 1:]  # Skip <SOS>

                # Find positions of EOW tokens (index 12)
                eow_positions = (target_seq == 12).nonzero(as_tuple=True)[0]

                # For each EOW, check the character right before it
                for eow_pos in eow_positions:
                    if eow_pos > 0:  # Make sure there's a character before EOW
                        last_char_pos = eow_pos - 1
                        
                        # Skip if it's a padding token
                        if target_seq[last_char_pos] != 13:
                            pred_last_char = pred_seq[last_char_pos]
                            target_last_char = target_seq[last_char_pos]
                            
                            # Check if prediction matches target
                            if pred_last_char == target_last_char:
                                correct_last_chars += 1
                            
                            total_last_chars += 1

    accuracy = (correct_last_chars / total_last_chars) * 100 if total_last_chars > 0 else 0
    error_rate = 100 - accuracy

    return accuracy, error_rate

In [ ]:
# ==================== LOAD MODEL AND EVALUATE ====================

def load_and_evaluate_last_char(model_path, val_loader, vocab):
    """Load trained model and calculate accuracy metrics"""

    # Hyperparameters (must match training)
    INPUT_VOCAB_SIZE = vocab.vocab_size_in
    OUTPUT_VOCAB_SIZE = vocab.vocab_size_out
    EMBED_SIZE = 512
    HIDDEN_SIZE = 512
    NUM_LAYERS = 2
    DROPOUT = 0.3

    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}\n")

    # Initialize model
    encoder = Encoder(INPUT_VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
    decoder = Decoder(OUTPUT_VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
    model = Seq2Seq(encoder, decoder, device).to(device)

    # Load checkpoint
    print(f"Loading model from: {model_path}")
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])

    print(f"Model loaded successfully!")
    print(f"Training epoch: {checkpoint['epoch']}")
    print(f"Training loss: {checkpoint['train_loss']:.4f}")
    print(f"Validation loss: {checkpoint['val_loss']:.4f}\n")

    # Calculate metrics
    print("=" * 60)
    print("Calculating Accuracy Metrics...")
    print("=" * 60)

    char_accuracy, der = calculate_last_character_accuracy(model, val_loader, device)
    # word_accuracy, wer = calculate_word_accuracy(model, val_loader, device)

    # Print results
    print("\n" + "=" * 60)
    print("EVALUATION RESULTS")
    print("=" * 60)
    print(f"Character Accuracy: {char_accuracy:.2f}%")
    print(f"DER (Diacritic Error Rate): {der:.2f}%")
    # print(f"\nWord Accuracy: {word_accuracy:.2f}%")
    # print(f"WER (Word Error Rate): {wer:.2f}%")
    print("=" * 60)

    return {
        'char_accuracy': char_accuracy,
        'der': der,
        # 'word_accuracy': word_accuracy,
        # 'wer': wer
    }

In [ ]:
model_path = 'best_diacritization_model.pt'

# Load and evaluate
results = load_and_evaluate_last_char(
    model_path=model_path,
    val_loader=val_loader_no_window,  # Your validation dataloader
    vocab = vocab
)